# Laboratorio 2

## Redes Neuronales

### Rodrigo Zea - 17058

# Introducción

El objetivo del laboratorio es desarrollar una red neuronal con el propósito de entrenar dicha red con un dataset de entrenamiento repleto de imagenes de artículos de ropa (camisas, vestidos, pantalones, bolsones, etc) para luego poder decir qué artículo es el observado. Por ejemplo, si se está analizando la imagen de una camisa, que la red neuronal, efectivamente, diga que es una camisa y no es confundido por otro artículo.

El dataset proveído es MNIST_Fashion, una alternativa al conocido MNIST, el cual es utilizado como benchmark para la mayoría de algoritmos de inteligencia artificial. MNIST_Fashion es una alternativa más compleja, debido a que muchos algoritmos sencillos pueden obtener un 97% de aciertos sin dificultad alguna en el MNIST original.

### Importación de librerias

In [1]:
import numpy as np
import mnist_reader
from functools import reduce
from scipy import optimize

### Algunos métodos de utilidades externos al algoritmo

Método para inflar matrices.

¿Para qué se usa? Se utiliza para "inflar" las matrices de las thetas luego de ser "aplanadas" en el algoritmo, entran al método como vectores "planos".

In [2]:
def inflate_matrixes(flat_thetas, shapes):
    layers = len(shapes) + 1
    sizes = [shape[0] * shape[1] for shape in shapes]
    steps = np.zeros(layers, dtype=int)

    for i in range(layers - 1):
        steps[i + 1] = steps[i] + sizes[i]

    return [
        flat_thetas[steps[i]: steps[i + 1]].reshape(*shapes[i])
        for i in range(layers - 1)
    ]

Función para aplanar matrices.

In [3]:
flatten_list_of_arrays = lambda list_of_arrays: reduce(
    lambda acc, v: np.array([*acc.flatten(), *v.flatten()]),
    list_of_arrays
)

Función sigmoide para transportar entradas por capa a salidas correspondientes.

In [4]:
# Basado en implementacion de https://towardsdatascience.com/fashion-product-image-classification-using-neural-networks-machine-learning-from-scratch-part-e9fda9e47661
sigmoid = lambda x: 1.0/(1.0 + np.exp(-x))

### Algoritmo de Redes Neuronales principal

Feed Forward, calcula la derivada (salidas) de cada capa neuronal por prueba.

In [5]:
def feed_forward(thetas, X):
    a = [X]

    for i in range(len(thetas)):
        a.append(
            sigmoid(
                np.matmul(
                    np.hstack((
                        np.ones(len(X)).reshape(len(X), 1),
                        a[i]
                    )), thetas[i].T
                )
            )            
        )
    return a

Función de costo

In [6]:
def cost_function(flat_thetas, shapes, X, Y):
    a = feed_forward(
        inflate_matrixes(flat_thetas, shapes),
        X
    )
    return -(Y * np.log(a[-1]) + (1 - Y) * np.log(1 - a[-1])).sum() / len(X)

Back propagation, calcula las matrices de Delta[i, j], es el descenso a gradiente.

In [7]:
def back_prop(flat_thetas, shapes, X, Y):
    m, layers = len(X), len(shapes) + 1
    thetas = inflate_matrixes(flat_thetas, shapes)
    
    a = feed_forward(thetas, X)

    # No existe necesidad de realizar un ciclo en estas operaciones aunque sean operaciones por capa
    # Esto es debido a que son operaciones que se pueden simplificar con operaciones matriciales.
    
    # 2.3 en algoritmo
    deltas = [*range(layers - 1), a[-1] - Y]
    
    # 2.4 en algoritmo
    for i in range(layers - 2, 0, -1):
        deltas[i] = np.matmul(deltas[i + 1], np.delete(thetas[i], 0, 1)) * (a[i] * (1 - a[i]))

    # 2.5 y 3
    Deltas = []
    for i in range(layers - 1):
        Deltas.append(
            ( 
            np.matmul(
                deltas[i + 1].T,
                np.hstack((
                    np.ones(len(a[i])).reshape(len(a[i]), 1),
                    a[i]))
                    )
            ) / m
        )
    Deltas = np.asarray(Deltas)

    return flatten_list_of_arrays(
        Deltas
    )

# Entrenamiento

#### Lectura de Datos

In [8]:
import mnist_reader
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

# reajuste
X_train = X_train/1000

#### Preperación de datos para pasos posteriores

In [9]:
m, n = X_train.shape
y_train = y_train.reshape(m, 1)
Y = (y_train == np.array(range(10))).astype(int)

#### Red neuronal principal

In [12]:
# Creación de red neuronal
# La cantidad de elementos en el array son las capas, y la cantidad de cada elemento es la cantidad de neuronas
NEURAL_NET = np.array([
    n,
    135,
    10
])

# Obtención de shapes de theta (matriz con pesos de transición)
theta_shapes = np.hstack((
    NEURAL_NET[1:].reshape(len(NEURAL_NET) - 1, 1),
    (NEURAL_NET[:-1] + 1).reshape(len(NEURAL_NET) - 1, 1)
))

# Se aplana theta 
flat_thetas = flatten_list_of_arrays([
    np.random.rand(*theta_shape) * 0.01
    for theta_shape in theta_shapes
])

In [13]:
# Encontramos los mejores thetas para nuestra red neuronal utilizando optimize de la libreria scipy
result = optimize.minimize(
    fun = cost_function,
    x0 = flat_thetas,
    args=(theta_shapes, X_train, Y),
    method='L-BFGS-B',
    jac= back_prop,
    options={'disp': True, 'maxiter': 3000}
)

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in multiply
  


#### Durante el procesamiento de datos, se desplegaron warnings debido a ciertas iteraciones que dieron como resultado NaN, sin embargo, el descenso a gradiente proveyó resultados válidos.

In [14]:
result

      fun: nan
 hess_inv: <107335x107335 LbfgsInvHessProduct with dtype=float64>
      jac: array([-3.12796721e-06, -1.21035090e-14, -7.37880961e-14, ...,
       -6.75105415e-09, -3.99548227e-08,  1.89774282e-10])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 9311
      nit: 1433
   status: 0
  success: True
        x: array([-3.1836563 ,  0.00622807,  0.00536195, ...,  0.59574482,
        1.66581616, -3.35844322])

In [31]:
thetaR = result.x
thetaR.size

107335

In [33]:
np.savetxt('thetasOptimal.txt', thetaR, delimiter=',')

In [34]:
thetaR = np.loadtxt('thetasOptimal.txt')

# Prueba

#### Lectura de Datos

In [36]:
X_test = X_test/1000
m,_ = X_test.shape
y_test = y_test.reshape(m,1)

#### Datos de resultados de theta (optimos)

In [37]:
flat_thetas_top = thetasR

#### Inflar thetas previamente aplanadas

In [38]:
superThetas = inflate_matrixes(flat_thetas_top,theta_shapes)

#### Varios pasos en uno

In [24]:
# Feed forward de prediccion
a = feed_forward(superThetas, X_test)

# Indices de porcentaje máximo
maximos = np.argmax(a[-1], axis = 1).reshape(m,1)
correct = ((maximos == y_test)*1).sum()

# Porcentajes de "precision" y errores
accuracy = (correct/m)*100
mistakes = 100.0 - accuracy

# Resultados de prueba

In [42]:
print("La precisión de acertados fue de: " + str(accuracy))

La precisión de acertados fue de: 87.03999999999999


In [43]:
print("El porcentaje de errores fue de: " + str(mistakes))

El porcentaje de errores fue de: 12.960000000000008


# Conclusiones

1. El modelo tiene un porcentaje de acertados relativamente alto y mayor al teórico (se esperaba que fuera 80%).
2. No tiene overfitting notable presente.
3. Al analizar los casos, la mayoría de casos de errores fue debido a prendas altamente parecidas. Por ejemplo, vestidos y sacos, o zapatos en general.